In [ ]:
import sys
sys.path.append('../')

In [2]:
from data.dataset import SegmentDataset
from data.collate import collate
from data.transforms import transform
from utils.vis import *
from utils.utils import *
from metrics.metrics import pixelAccuracy

import numpy as np
import torch

In [3]:
dataset = SegmentDataset(
    annot='../assets/train_sample.txt', 
    transform=transform, 
    dim=(2048, 2048),
)

In [7]:
min_acc, max_acc, tot_acc, tot_anomaly_acc = 1.0, 0.0, 0.0, 0.0
n_anomalies = 0

for idx, (_, mask_path, _, _) in enumerate(dataset):
    
    mask_original = processMask(
        mask_path,
        use_path=True,
        bake_anomaly=False,
        ret='arr',
    )
    mask_new = processMask(
        mask_path,
        use_path=True,
        bake_anomaly=True,
        ret='arr',
    )
    acc = pixelAccuracy(mask_original, mask_new)
    min_acc = min(acc, min_acc)
    max_acc = max(acc, max_acc)
    tot_acc += acc
    if acc!=1.0:
        tot_anomaly_acc += acc
        n_anomalies += 1
    print("acc : {:.6f}, min_acc : {:.6f}, max_acc : {:.6f}, avg_acc : {:.6f}, avg anomaly acc : {:.6f}"
          .format(acc, min_acc, max_acc, tot_acc/(idx+1), tot_anomaly_acc/n_anomalies), end='\r')